In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.005387544631958008
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 25.643311603713467
200000 36.05477452830729
300000 36.82302861760447
400000 41.71950844687519
500000 38.676985380049324
600000 36.040668566744664
700000 42.664439028325
800000 37.45944163965151
900000 34.52951467367252
1000000 31.966553365699532
1100000 34.25373849043171
1200000 31.960948168326368
1300000 17.75159966823841
1400000 24.46926999960726
1500000 34.00777722253623
1600000 32.43877162195686
1700000 34.2860473805626
1800000 32.15879111307746
1900000 34.93118042508594
2000000 34.19790763238509
2100000 34.8092142378642
2200000 34.45416823896203
2300000 33.13816787939174
2400000 35.67247877784481
2500000 32.76408726490426
2600000 15.063102605607789
2700000 28.4637864367388
2800000 43.41916452761684
2900000 42.95589470052874
3000000 42.70463837056941
3

29800000 33.567098150606924
29900000 32.477631035049896
30000000 34.936559696287766
30100000 33.63323530708525
30200000 34.36313187741403
30300000 35.51732354952257
30400000 34.707242851723684
30500000 35.579994054176545
30600000 36.427155221784595
30700000 13.19363154239566
30800000 37.62831529754278
30900000 34.52192712060451
31000000 36.133785994862386
31100000 20.524807040819947
31200000 26.62295656361681
31300000 34.562432297599145
31400000 34.0890219934163
31500000 35.65876319329543
31600000 32.3531260038367
31700000 36.14859594225362
31800000 34.64113277213221
31900000 31.216696591221524
32000000 33.652086929360564
32100000 31.356040278270836
32200000 8.96449323446504
32300000 31.872285994458863
32400000 33.24126645406211
32500000 32.27949922260527
32600000 33.74058872413997
32700000 34.432072182128394
32800000 31.30604009701055
32900000 34.82986806539392
33000000 35.036263459400615
33100000 32.80377674717251
33200000 22.749107599544985
33300000 25.60089242758356
33400000 35.992

59700000 24.559289674048074
59800000 31.159613163184947
59900000 30.57476083719828
60000000 29.0322609001486
60100000 29.661427913832444
60200000 28.537959822023595
60300000 28.452625439186907
60400000 25.225819677870845
60500000 11.730731389797363
60600000 26.90295853089435
60700000 30.042290665652793
60800000 29.497793073906376
60900000 28.258995032673976
61000000 30.832942190345626
61100000 28.19539935111285
61200000 31.098939212670302
61300000 13.791383196892149
61400000 29.839205534656784
61500000 31.610982431689198
61600000 29.678122247413114
61700000 32.73773461946165
61800000 29.7777561672363
61900000 31.31658287612744
62000000 31.54524396885092
62100000 30.033227192964894
62200000 15.753894920615707
62300000 25.52628888678272
62400000 31.021108832950237
62500000 28.060985050923367
62600000 30.83023951615817
62700000 29.2777182819382
62800000 26.73692319172172
62900000 28.190154809959278
63000000 16.107401666297108
63100000 19.070080320329243
63200000 27.620638119952144
6330000

89400000 9.064443421551921
89500000 27.467504305082368
89600000 26.929081198746342
89700000 26.731180115671552
89800000 27.311700429270203
89900000 27.615227906042925
90000000 27.79105457889441
90100000 26.18018686614093
90200000 27.136348924749257
90300000 26.50304853697178
90400000 26.165443246329225
90500000 25.662666583405393
90600000 26.03027045638775
90700000 10.258337331457968
90800000 23.172419240368406
90900000 24.973608998311107
91000000 23.77552225665405
91100000 25.140746470596387
91200000 25.05130609919137
91300000 11.55954906071319
91400000 19.148166547234638
91500000 25.484538287557942
91600000 24.503181762512398
91700000 24.50380667119115
91800000 24.90580383191672
91900000 17.336845583810728
92000000 18.867989451477456
92100000 26.1597070116984
92200000 26.576759710796154
92300000 27.556400214068205
92400000 27.289907438275964
92500000 27.460060523217507
92600000 26.97391318461331
92700000 27.316873329622812
92800000 26.98530870512379
92900000 27.609606519629732
930000

118500000 23.889828829534636
118600000 24.431023406104284
118700000 24.26706658113099
118800000 22.657988969705805
118900000 22.521827881709466
119000000 21.844257203529494
119100000 22.117769059125365
119200000 6.913681006857264
119300000 22.234044933098204
119400000 22.185343721094117
119500000 21.305492213288098
119600000 7.5613505839941375
119700000 20.131506422985623
119800000 21.06891895250322
119900000 22.116997735109656
120000000 12.00102747975996
120100000 18.765042815646368
120200000 22.11783875729418
120300000 21.77064726912478
120400000 21.590273510768466
120500000 23.203486998299685
120600000 23.610463356990888
120700000 23.94913908251403
120800000 23.734240449565057
120900000 23.85131585163552
121000000 24.691982839391223
121100000 24.19887878326222
121200000 23.96551867198464
121300000 24.24338059581635
121400000 22.656083732923218
121500000 22.26000457069635
121600000 21.888825331543302
121700000 22.612581407711
121800000 6.767486630925486
121900000 21.931297888188748
1

...

233200000 5.9078117555683605
233300000 11.448471783015023
233400000 11.35739356604416
233500000 12.018187031250411
233600000 11.992965716745983
233700000 12.071995972610914
233800000 12.649527520173903
233900000 12.368032943703708
234000000 12.51941863440962
234100000 12.47687594820195
234200000 12.436477584283034
234300000 12.518882541410651
234400000 12.25527434077304
234500000 11.377028800066418
234600000 11.625215195743035
234700000 11.004030818632808
234800000 11.477526672471164
234900000 4.054669152635393
235000000 11.59217274371882
235100000 11.056541286560897
235200000 11.350989379708277
235300000 4.78369300059213
235400000 9.526139062730989
235500000 11.171544827921153
235600000 10.924622063583408
235700000 8.433260250211168
235800000 6.894787070463968
235900000 10.882282088251236
236000000 11.243732930543455
236100000 11.710016448271682
236200000 11.83333115562737
236300000 12.050176819547104
236400000 12.487930557480942
236500000 12.320784633208964
236600000 12.1884087448589

262100000 9.649300948076373
262200000 10.241659664138682
262300000 9.017489524602098
262400000 9.804794214622822
262500000 9.611637190886853
262600000 9.6094770651525
262700000 9.449660422653672
262800000 9.075963741199683
262900000 7.910278354400587
263000000 4.6702811939191715
263100000 8.450524902670772
263200000 8.548022269356482
263300000 8.444704094914353
263400000 8.412712121848548
263500000 8.689837991438825
263600000 2.4215120693142076
263700000 8.317132558974933
263800000 8.680468799226983
263900000 8.797355189985044
264000000 8.511524194670793
264100000 8.619677086857957
264200000 5.9051154822475285
264300000 6.087338489227478
264400000 9.03029445869438
264500000 9.170269814327646
264600000 9.36913070539326
264700000 9.441004878374386
264800000 9.962497404972865
264900000 9.915204885248569
265000000 9.797856395706136
265100000 9.505625570058937
265200000 9.672288667906473
265300000 9.240441774191202
265400000 9.164587816574175
265500000 4.668084293628805
265600000 7.41313436

291400000 6.280515251649779
291500000 5.984507791256312
291600000 6.082147635536517
291700000 5.940747578546832
291800000 6.020721050305241
291900000 1.132065989801732
292000000 6.172429407210725
292100000 5.978420488571781
292200000 6.001506200468654
292300000 5.8822968569274225
292400000 6.211474854569673
292500000 6.1261739805701785
292600000 6.233156332532535
292700000 3.8793685562951064
292800000 4.619108224807284
292900000 6.406989949799108
293000000 6.305688669768172
293100000 6.410624990492866
293200000 6.431754099817273
293300000 6.4382907592218315
293400000 6.399845330218871
293500000 6.341701722822909
293600000 2.1406863487287553
293700000 6.728495087626438
293800000 6.07867662214995
293900000 5.870059966488542
294000000 5.9114492309032
294100000 5.703416901588595
294200000 5.613789004514386
294300000 5.616800375804087
294400000 4.433478890213263
294500000 2.806980309392189
294600000 5.524809539323914
294700000 5.489583743887032
294800000 5.592048139040904
294900000 6.230860

320600000 3.6480787390883105
320700000 3.867318053751337
320800000 3.4006923744030595
320900000 3.727437579051832
321000000 3.5929280500279925
321100000 3.6167602776292944
321200000 2.0712988186342605
321300000 2.854662104239221
321400000 3.8667631248640832
321500000 3.4731639132331678
321600000 3.4712117896049244
321700000 1.532564508971655
321800000 3.647772193547397
321900000 3.595740673314338
322000000 3.51901647154962
322100000 3.5617345369629883
322200000 3.298125912948465
322300000 3.5227035204678336
322400000 3.5092671781057665
322500000 3.260550522462747
322600000 3.3424241533735692
322700000 2.055172006121328
322800000 2.150037647071446
322900000 3.1320306437819223
323000000 3.4417238201867995
323100000 3.4014656201574707
323200000 3.2632659372109494
323300000 3.3215557949059793
323400000 3.2697466291197896
323500000 3.342550291494721
323600000 3.401088475955809
323700000 3.2580172497430064
323800000 1.714230241466822
323900000 2.8832490360457803
324000000 3.4159223658626234


349200000 0.3299106721503284
349300000 0.34884059605603224
349400000 0.3368244821313606
349500000 0.3228472450688064
349600000 0.32263996298046116
349700000 0.16258756305047803
349800000 0.1510958439276119
349900000 0.2515624816923221
350000000 0.23363674402686752
350100000 0.2295796057668315
350200000 0.22691842538398904
350300000 0.24252020199776161
350400000 0.2236799290747272
350500000 0.21362166618312597
350600000 0.21485859161544638
350700000 0.19989719494166774
350800000 0.19891820987534323
350900000 0.18313077010228435
351000000 0.10028880777571665
351100000 0.12852374115370116
351200000 0.1958124265404675
351300000 0.1714462742354466
351400000 0.16934465438855292
351500000 0.16031030608651636
351600000 0.15267665908645325
351700000 0.17018807044363418
351800000 0.15381157246266272
351900000 0.14597117222765751
352000000 0.1444789234824538
352100000 0.12960030836925043
352200000 0.12544011572662328
best so far: 0
type: [1, 1, 1, 137, 1] 137
cases of this type: 2571353
352300000